In [1]:
import pandas as pd

# = = =  = = =  = = =

from selenium.webdriver import ChromeOptions

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from selenium.webdriver import Chrome
from selenium.webdriver.chrome.service import Service

import time

# = = =  = = =  = = =

from urllib.parse import urlparse, parse_qs, urlencode

from bs4 import BeautifulSoup
from lxml import etree

# = = =  = = =  = = =

import re

# = = =  = = =  = = =

import json

# = = =  = = =  = = =

import html as html_lib

In [2]:
print('Data Loading...')
list_dict_correct, list_series_error, df_input = [], [], pd.read_excel('./file/1.url.xlsx',
                                                                       header=0,
                                                                       dtype=str).fillna('')
print('Loading Done !')
print()

df_input['No'] = df_input['No'].astype(int)

crawler_remain = len(df_input)

print(f'总数量：{crawler_remain}')
print()

df_input

Data Loading...
Loading Done !

总数量：100



,No,Url
0,1,https://www.amazon.com/dp/B08W98H7H7
1,2,https://www.amazon.com/dp/B093S8XVSL
2,3,https://www.amazon.com/dp/B08W9MX59Y
3,4,https://www.amazon.com/dp/B08TTC1HQ3
4,5,https://www.amazon.com/dp/B0BPCP4XQN
...,...,...
95,96,https://www.amazon.com/dp/B0BT22K3G7
96,97,https://www.amazon.com/dp/B0BH4ZXNKP
97,98,https://www.amazon.com/dp/B0BY213GX3
98,99,https://www.amazon.com/dp/B095C2ZSK1


In [3]:
option = ChromeOptions()
option.add_experimental_option('useAutomationExtension', False)
option.add_experimental_option('excludeSwitches', ['enable-automation'])
option.add_argument('--disable-blink-features=AutomationControlled')
option.add_argument('--disable-dev-shm-usage')
option.add_argument('--disable-gpu')
option.add_argument('--no-sandbox')
# option.add_argument('--headless=new')
# option.add_argument('blink-settings=imagesEnabled=false')
option.add_argument(get_header(method='selenium'))
option.add_argument(get_proxy(method='selenium'))
option.page_load_strategy = 'normal'

driver = Chrome(service=Service('../chromedriver'), options=option)
driver.maximize_window()

# = = =  = = =  = = =

crawler_retry = 0
while True:
    crawler_retry += 1
    print(crawler_retry)
    try:
        driver.get('https://www.amazon.com/?language=en_US')
        time.sleep(1)
        driver.execute_script("window.scrollTo(0, 0.2 * document.body.scrollHeight);")
        time.sleep(1.5)
        driver.execute_script("window.scrollTo(0, 0.45 * document.body.scrollHeight);")
        time.sleep(0.7)
        driver.execute_script("window.scrollTo(0, 0.7 * document.body.scrollHeight);")
        time.sleep(1.2)
        driver.execute_script("window.scrollTo(0, 0.95 * document.body.scrollHeight);")
        time.sleep(1)
        driver.execute_script("window.scrollTo(0, 0);")

        break
    except KeyboardInterrupt:
        break
    except:
        time.sleep(3)
        continue

driver

1


<selenium.webdriver.chrome.webdriver.WebDriver (session="3573e73d6486d82a1d94a3e1dcb7de09")>

In [4]:
crawler_status, (crawler_index, crawler_series) = False, next(df_input.iterrows())

crawler_status, crawler_index, crawler_series

(False,
 0,
 No                                        1
 Url    https://www.amazon.com/dp/B08W98H7H7
 Name: 0, dtype: object)

In [5]:
url_parse = urlparse(crawler_series['Url'])

dict_param = parse_qs(url_parse.query)
dict_param['language'] = 'en_US'

url_request = f'{url_parse.scheme}://{url_parse.netloc}{url_parse.path}?{urlencode(dict_param, doseq=True)}'

crawler_retry = 0
while True:
    crawler_retry += 1

    try:
        driver.get(url_request)
        time.sleep(1)
        driver.execute_script("window.scrollTo(0, 0.2 * document.body.scrollHeight);")
        time.sleep(1.5)
        driver.execute_script("window.scrollTo(0, 0.45 * document.body.scrollHeight);")
        time.sleep(0.7)
        driver.execute_script("window.scrollTo(0, 0.7 * document.body.scrollHeight);")
        time.sleep(1.2)
        driver.execute_script("window.scrollTo(0, 0.95 * document.body.scrollHeight);")
        time.sleep(1)
        driver.execute_script("window.scrollTo(0, 0);")

        soup = BeautifulSoup(driver.page_source, 'lxml')
        html = etree.HTML(str(soup))

        with open('./part.html', 'w', encoding='UTF-8') as file:
            file.write(str(soup))

        break
    except KeyboardInterrupt:
        break
    except:
        continue

url_request

'https://www.amazon.com/dp/B08W98H7H7?language=en_US'

In [6]:
zip_code = re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', html.xpath('//span[@id="glow-ingress-line2"]/text()')[0]).strip()

zip_code

'Chino 91710'

In [7]:
price = html.xpath('//span[@class="a-offscreen"]/text()')[0].strip() if html.xpath('//span[@class="a-offscreen"]/text()') else ''

price

'$59.99'

In [8]:
dict_src, list_src = {}, json.loads('{"initial":[{' + str(soup).split('\'colorImages\': { \'initial\': [{')[1].split('}]},')[0] + '}]}')['initial']
for i, src in enumerate(list_src):
    dict_src[str(i)] = src['hiRes'].strip() if src['hiRes'] else src['large'].strip()

dict_src

{'0': 'https://m.media-amazon.com/images/I/71wmhb446HL._AC_SL1500_.jpg',
 '1': 'https://m.media-amazon.com/images/I/61MmawXIPpL._AC_SL1500_.jpg',
 '2': 'https://m.media-amazon.com/images/I/61SQ4nZeAsL._AC_SL1500_.jpg',
 '3': 'https://m.media-amazon.com/images/I/719cpaejDhL._AC_SL1500_.jpg',
 '4': 'https://m.media-amazon.com/images/I/71F+7vhZHgL._AC_SL1500_.jpg',
 '5': 'https://m.media-amazon.com/images/I/71vEvtWLDZL._AC_SL1500_.jpg',
 '6': 'https://m.media-amazon.com/images/I/71zda1Q0fhL._AC_SL1500_.jpg',
 '7': 'https://m.media-amazon.com/images/I/71PUp0xK1pL._AC_SL1500_.jpg'}

In [9]:
dict_key, list_tr = {}, html.xpath('//table[@class="a-normal a-spacing-micro"]/tbody/tr')
for tr in list_tr:
    key = re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./td/span/text()')[0]).strip()
    if key in dict_key:
        dict_key[key] += 1
    else:
        dict_key[key] = 1

dict_overview, dict_count = {}, {}
for i, tr in enumerate(list_tr):
    key = re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./td/span/text()')[0]).strip()
    if dict_key[key] == 1:
        dict_overview[str(i)] = {key: re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./td/span/text()')[1]).strip()}
    else:
        if key in dict_key:
            dict_key[key] += 1
        else:
            dict_key[key] = 1
        dict_overview[str(i)] = {f'{key}-{dict_count[key]}': re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./td/span/text()')[1]).strip()}

dict_overview

{'0': {'Brand': 'A-Premium'},
 '1': {'Voltage': '12 Volts'},
 '2': {'Material': 'Metal,Plastic'},
 '3': {'Item Weight': '1.54 Pounds'},
 '4': {'Manufacturer': 'PremiumpartsWhosale'}}

In [10]:
about, list_text = '', [re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', text).strip() for text in html.xpath('//ul[@class="a-unordered-list a-vertical a-spacing-mini"]/li/span/text()')]
for i, text in enumerate(list_text):
    about += f'{i + 1}. {text}\n'

print(about.strip())

1. [Vehicle Fitment]: Compatible with Ford Explorer 2011-2017 Sport Utility (Before 2/11/2017) --- No Tilt Function
2. [Specifications]: ① Position: (Sunroof) Left or Right; ② Terminal Quantity: 14 Pins; ③ Attachment Type: Screw-in; ④ Electric/Manual: Electric; ⑤ Material: Metal and Plastic; ⑥ Installation Hardware: with 3 Screws; ⑦ Fitment: Direct Replacement
3. [Reference Number]: BB5Z15790A, BB5Z15790D, BB5Z-15790-A, BB5Z-15790-D, MM-1147, MM-1038
4. [Features]: A-Premium Sunroof Motor is engineered to provide OE fit, form, and function. Being durable and wear-resistant, it ensures strong reliability, coupled with high technology to improve operating efficiency and response speed, and make the air fresh in the car.
5. [Buy with Confidence]: A-Premium offers a one-year unlimited-mileage guarantee on our meticulously crafted sunroof motors. We offer a wide variety of automotive accessory categories, ensuring that you can hit the road with peace of mind.


In [11]:
dict_ = json.loads(re.sub(r'\\(?!["\\])', r'\\\\', str(soup).split('jQuery.parseJSON(\'')[1].split('}\');')[0] + '}'))

dict_

{'dataInJson': None,
 'alwaysIncludeVideo': True,
 'autoplayVideo': False,
 'defaultColor': 'initial',
 'mainImageSizes': [['355', '355'],
  ['450', '450'],
  ['425', '550'],
  ['466', '606'],
  ['522', '679'],
  ['569', '741'],
  ['679', '879']],
 'maxAlts': 7,
 'altsOnLeft': True,
 'productGroupID': 'automotive_display_on_website',
 'lazyLoadExperienceDisabled': True,
 'lazyLoadExperienceOnHoverDisabled': False,
 'useChromelessVideoPlayer': False,
 'colorToAsin': {},
 'refactorEnabled': True,
 'useIV': True,
 'tabletWeb': False,
 'views': ['ImageBlockMagnifierView',
  'ImageBlockAltImageView',
  'ImageBlockVideoView',
  'ImageBlockTwisterView',
  'ImageBlockImmersiveViewImages',
  'ImageBlockImmersiveViewVideos',
  'ImageBlockImmersiveViewDimensionIngress',
  'ImageBlockImmersiveViewShowroom',
  'ImageBlockImmersiveView360',
  'ImageBlockTabbedImmersiveView',
  'ImageBlockShoppableSceneView'],
 'enhancedHoverOverlay': False,
 'landingAsinColor': 'initial',
 'colorImages': {},
 'heroI

In [12]:
title = re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', dict_['title']).strip()

title

'A-Premium Sunroof Moon Roof Motor Compatible with Ford Explorer 2011-2017 Sport Utility BB5Z15790A Left or Right'

In [13]:
parent_asin = dict_['parentAsin'].strip()

parent_asin

'B08W98H7H7'

In [14]:
dict_key, list_tr = {}, html.xpath('//table[@id="productDetails_techSpec_section_1"]/tbody/tr')
for tr in list_tr:
    key = re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./th/text()')[0]).strip()
    if key in dict_key:
        dict_key[key] += 1
    else:
        dict_key[key] = 1

dict_technical, dict_count = {}, {}
for i, tr in enumerate(list_tr):
    key = re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./th/text()')[0]).strip()
    if dict_key[key] == 1:
        dict_technical[str(i)] = {key: re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./td/text()')[0]).strip()}
    else:
        if key in dict_count:
            dict_count[key] += 1
        else:
            dict_count[key] = 1
        dict_technical[str(i)] = {f'{key}-{dict_count[key]}': re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./td/text()')[0]).strip()}

dict_technical

{'0': {'Manufacturer': 'PremiumpartsWhosale'},
 '1': {'Brand': 'A-Premium'},
 '2': {'Item Weight': '1.54 pounds'},
 '3': {'Package Dimensions': '7.8 x 4.84 x 2.13 inches'},
 '4': {'Item model number': 'Compatible with Ford'},
 '5': {'Manufacturer Part Number': 'APSRM001'},
 '6': {'OEM Part Number': 'BB5Z15790A, BB5Z15790D, BB5Z-15790-A, BB5Z-15790-D, MM-1147, MM-1038'},
 '7': {'Voltage': '12 Volts'}}

In [15]:
i, dict_additional, list_tr = 0, {}, html.xpath('//table[@id="productDetails_detailBullets_sections1"]/tbody/tr')
for tr in list_tr:
    key = re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./th/text()')[0]).strip()
    if key == 'Customer Reviews':
        dict_additional[str(i)] = {'Customer Reviews Rating Count': tr.xpath('./td/div/span[@data-action="acrLink-click-metrics"]/a/span/text()')[0].strip()}
        dict_additional[str(i + 1)] = {'Customer Reviews Rating': tr.xpath('./td/div/span[@data-action="acrStarsLink-click-metrics"]/span/@title')[0].strip()}
        i += 2
    elif key == 'Best Sellers Rank':
        list_span = tr.xpath('./td/span/span')
        list_text = [etree.tostring(span).decode('utf-8') for span in list_span]
        list_text = [re.sub(r'<.*?>|\(.*?\)', '', text).strip() for text in list_text]
        list_text = [re.sub(r' +', ' ', text.strip()) for text in list_text]
        list_text = [html_lib.unescape(text) for text in list_text]

        list_href = [f'https://www.amazon.com{href.strip()}' for href in tr.xpath('./td/span/span/a/@href')]
        for j, (text, href) in enumerate(zip(list_text, list_href)):
            if j == 0:
                dict_additional[str(i)] = {'Best Sellers Rank': text}
                dict_additional[str(i + 1)] = {'Best Sellers Rank Url': href}
            else:
                dict_additional[str(i)] = {f'Best Sellers Rank-{j + 1}': text}
                dict_additional[str(i + 1)] = {f'Best Sellers Rank-{j + 1} Url': href}
            i += 2
    else:
        dict_additional[str(i)] = {key: re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./td/text()')[0]).strip()}
        i += 1

dict_additional

{'0': {'ASIN': 'B08W98H7H7'},
 '1': {'Customer Reviews Rating Count': '244 ratings'},
 '2': {'Customer Reviews Rating': '3.9 out of 5 stars'},
 '3': {'Best Sellers Rank': '#34,287 in Automotive'},
 '4': {'Best Sellers Rank Url': 'https://www.amazon.com/gp/bestsellers/automotive/ref=pd_zg_ts_automotive'},
 '5': {'Best Sellers Rank-2': '#1 in Automotive Replacement Sun Roof Motors'},
 '6': {'Best Sellers Rank-2 Url': 'https://www.amazon.com/gp/bestsellers/automotive/15730511/ref=pd_zg_hrsr_automotive'},
 '7': {'Date First Available': 'February 8, 2021'}}

In [16]:
dict_key, list_tr = {}, html.xpath('//div[@id="detailBulletsWrapper_feature_div"]/div[@id="detailBullets_feature_div"]/ul/li/span')
for tr in list_tr:
    key = re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./span/text()')[0]).strip().removesuffix(':').strip()
    if key in dict_key:
        dict_key[key] += 1
    else:
        dict_key[key] = 1

dict_technical_2, dict_count = {}, {}
for i, tr in enumerate(list_tr):
    key = re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./span/text()')[0]).strip().removesuffix(':').strip()
    if dict_key[key] == 1:
        dict_technical_2[str(i)] = {key: re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./span/text()')[1]).strip()}
    else:
        if key in dict_count:
            dict_count[key] += 1
        else:
            dict_count[key] = 1
        dict_technical_2[str(i)] = {f'{key}-{dict_count[key]}': re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./span/text()')[1]).strip()}

dict_technical_2

{}

In [17]:
i, dict_additional_2, list_span = 0, {}, html.xpath('//div[@id="detailBulletsWrapper_feature_div"]/ul[@class="a-unordered-list a-nostyle a-vertical a-spacing-none detail-bullet-list"]/li/span')
for span in list_span:
    key = re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', span.xpath('./span/text()')[0]).strip().removesuffix(':').strip()
    if key == 'Customer Reviews':
        dict_additional_2[str(i)] = {'Customer Reviews Rating Count': span.xpath('./div/span[@data-action="acrLink-click-metrics"]/a/span/text()')[0].strip()}
        dict_additional_2[str(i + 1)] = {'Customer Reviews Rating': span.xpath('./div/span[@data-action="acrStarsLink-click-metrics"]/span/@title')[0].strip()}
        i += 2
    elif key == 'Best Sellers Rank':
        list_span_2 = span.xpath('./descendant-or-self::span[@class="a-list-item"]')
        list_text = [etree.tostring(span_2).decode('utf-8') for span_2 in list_span_2]
        list_text = [re.sub(r'<ul.*?</ul>|<span.*?Best Sellers Rank.*?</span>|<.*?>|\(.*?\)', '', text).strip() for text in list_text]
        list_text = [re.sub(r' +', ' ', text.strip()) for text in list_text]
        list_text = [html_lib.unescape(text) for text in list_text]

        list_href = [f'https://www.amazon.com{href.strip()}' for href in span.xpath('./descendant-or-self::a/@href')]
        for j, (text, href) in enumerate(zip(list_text, list_href)):
            if j == 0:
                dict_additional_2[str(i)] = {'Best Sellers Rank': text}
                dict_additional_2[str(i + 1)] = {'Best Sellers Rank Url': href}
            else:
                dict_additional_2[str(i)] = {f'Best Sellers Rank-{j + 1}': text}
                dict_additional_2[str(i + 1)] = {f'Best Sellers Rank-{j + 1} Url': href}
            i += 2
    else:
        print('fuck')

dict_additional_2

{}

In [18]:
dict_key, list_tr = {}, html.xpath('//div[@id="productDetails_expanderSectionTables"]/div[@class="a-row"]/div[contains(@class, "a-column a-span6")]/div[@class="a-row a-spacing-medium-plus"]/div[@class="a-section"]/div[contains(@data-csa-c-content-id, "voyager-expander-btn")]/div/div[@class="a-section"]/table/tbody/tr')
for tr in list_tr:
    key = re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./th/text()')[0]).strip().removesuffix(':').strip()
    if key in dict_key:
        dict_key[key] += 1
    else:
        dict_key[key] = 1

i, dict_information, dict_count = 0, {}, {}
for tr in list_tr:
    key = re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./th/text()')[0]).strip().removesuffix(':').strip()
    if dict_key[key] == 1:
        if key == 'Customer Reviews':
            dict_information[str(i)] = {'Customer Reviews Rating Count': tr.xpath('./td/div/span[@data-action="acrLink-click-metrics"]/a/span/text()')[0].strip()}
            dict_information[str(i + 1)] = {'Customer Reviews Rating': tr.xpath('./td/div/span[@data-action="acrStarsLink-click-metrics"]/span/@title')[0].strip()}
            i += 2
        elif key == 'Best Sellers Rank':
            list_span = tr.xpath('./td/span/span')
            list_text = [etree.tostring(span).decode('utf-8') for span in list_span]
            list_text = [re.sub(r'<.*?>|\(.*?\)', '', text).strip() for text in list_text]
            list_text = [re.sub(r' +', ' ', text.strip()) for text in list_text]
            list_text = [html_lib.unescape(text) for text in list_text]

            list_href = [f'https://www.amazon.com{href.strip()}' for href in tr.xpath('./td/span/span/a/@href')]
            for j, (text, href) in enumerate(zip(list_text, list_href)):
                if j == 0:
                    dict_information[str(i)] = {'Best Sellers Rank': text}
                    dict_information[str(i + 1)] = {'Best Sellers Rank Url': href}
                else:
                    dict_information[str(i)] = {f'Best Sellers Rank-{j + 1}': text}
                    dict_information[str(i + 1)] = {f'Best Sellers Rank-{j + 1} Url': href}
                i += 2
        else:
            dict_information[str(i)] = {key: re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./td/text()')[0]).strip()}
            i += 1
    else:
        if key in dict_count:
            dict_count[key] += 1
        else:
            dict_count[key] = 1
        dict_information[str(i)] = {f'{key}-{dict_count[key]}': re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./td/text()')[0]).strip()}

dict_information

{}

In [19]:
not dict_technical and not dict_additional and not dict_technical_2 and not dict_additional_2 and not dict_information

False

In [20]:
list_dict_correct.append({'No': crawler_series['No'],
                          'Title': title,
                          'About': about.strip(),
                          'Price': price,
                          'Picture': '',
                          'Url': url_request,
                          'Zip Code': zip_code,
                          'Parent ASIN': parent_asin,
                          'Json_Src': json.dumps(dict_src),
                          'Json_Overview': json.dumps(dict_overview),
                          'Json_Technical': json.dumps(dict_technical),
                          'Json_Additional': json.dumps(dict_additional),
                          'Json_Technical 2': json.dumps(dict_technical_2),
                          'Json_Additional 2': json.dumps(dict_additional_2),
                          'Json_Information': json.dumps(dict_information)})

list_dict_correct

[{'No': 1,
  'Title': 'A-Premium Sunroof Moon Roof Motor Compatible with Ford Explorer 2011-2017 Sport Utility BB5Z15790A Left or Right',
  'About': '1. [Vehicle Fitment]: Compatible with Ford Explorer 2011-2017 Sport Utility (Before 2/11/2017) --- No Tilt Function\n2. [Specifications]: ① Position: (Sunroof) Left or Right; ② Terminal Quantity: 14 Pins; ③ Attachment Type: Screw-in; ④ Electric/Manual: Electric; ⑤ Material: Metal and Plastic; ⑥ Installation Hardware: with 3 Screws; ⑦ Fitment: Direct Replacement\n3. [Reference Number]: BB5Z15790A, BB5Z15790D, BB5Z-15790-A, BB5Z-15790-D, MM-1147, MM-1038\n4. [Features]: A-Premium Sunroof Motor is engineered to provide OE fit, form, and function. Being durable and wear-resistant, it ensures strong reliability, coupled with high technology to improve operating efficiency and response speed, and make the air fresh in the car.\n5. [Buy with Confidence]: A-Premium offers a one-year unlimited-mileage guarantee on our meticulously crafted sunroof 

In [21]:
crawler_status = True

crawler_status

True

In [22]:
crawler_series['Request_Url'] = url_request

list_series_error.append(crawler_series)

list_series_error

[No                                                             1
 Url                         https://www.amazon.com/dp/B08W98H7H7
 Request_Url    https://www.amazon.com/dp/B08W98H7H7?language=...
 Name: 0, dtype: object]

In [23]:
print('Data Outputting...')
if list_dict_correct:
    df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['No'],
                                                             ascending=[True],
                                                             ignore_index=True)
    df_correct.to_excel('./test_part.xlsx', index=False)

df_correct if list_dict_correct else ''

Data Outputting...


,No,Title,About,Price,Picture,Url,Zip Code,Parent ASIN,Json_Src,Json_Overview,Json_Technical,Json_Additional,Json_Technical 2,Json_Additional 2,Json_Information
0,1,A-Premium Sunroof Moon Roof Motor Compatible w...,1. [Vehicle Fitment]: Compatible with Ford Exp...,$59.99,,https://www.amazon.com/dp/B08W98H7H7?language=...,Chino 91710,B08W98H7H7,"{""0"": ""https://m.media-amazon.com/images/I/71w...","{""0"": {""Brand"": ""A-Premium""}, ""1"": {""Voltage"":...","{""0"": {""Manufacturer"": ""PremiumpartsWhosale""},...","{""0"": {""ASIN"": ""B08W98H7H7""}, ""1"": {""Customer ...",{},{},{}


In [24]:
print('Output Finished !')
print()

Output Finished !



In [25]:
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                           ascending=[True],
                                                           ignore_index=True)
    df_error.to_excel('./test_part_error.xlsx', index=False)
    print('- Error Occurred -')
    print()

df_error if list_series_error else ''

- Error Occurred -



,No,Url,Request_Url
0,1,https://www.amazon.com/dp/B08W98H7H7,https://www.amazon.com/dp/B08W98H7H7?language=...


In [26]:
driver.quit()
print('Done ~')

Done ~
